In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime, timedelta

In [2]:
# 自訂 function 可重新 reload
from importlib import reload
from utils import util_inference_result
reload(util_inference_result)
from utils.util_inference_result import *

In [7]:
# NYC 目前只有一筆
# main_folder = '/home/hchuang/Documents/Project/SSSD_CP/src/'
# result_folder = 'results/NYISO_NYC/T200_beta00.0001_betaT0.02/'
# dataset_folder = 'datasets/NYISO/dataset_1/zone/'
# dataset_filename = 'load_array_N.Y.C._test.npy'

# NYC_split
# main_folder = '/home/hchuang/Documents/Project/SSSD_CP/src/'
# result_folder = 'results/NYISO_NYC/T200_beta00.0001_betaT0.02/'
# dataset_folder = 'datasets/NYISO/dataset_1/zone/'
# dataset_filename = 'load_array_N.Y.C._test.npy'

# mix
# main_folder = '/home/hchuang/Documents/Project/SSSD_CP/src/'
# result_folder = 'results/NYISO_mix/T200_beta00.0001_betaT0.02/imputaiton_multiple_mix_160k/'
# dataset_folder = 'datasets/NYISO/dataset_2/mix/'
# dataset_filename = 'mix_test_data_0.npy'


# mix_fine_tune_1
# main_folder = '/home/hchuang/Documents/Project/SSSD_CP/src/'
# result_folder = 'results/NYISO_mix_fine_tune_1/T200_beta00.0001_betaT0.02/imputaiton_multiple_162k/'
# dataset_folder = 'datasets/NYISO/dataset_2/mix/'
# dataset_filename = 'mix_test_data_1.npy'

# mix_fine_tune_2
# main_folder = '/home/hchuang/Documents/Project/SSSD_CP/src/'
# result_folder = 'results/NYISO_mix_fine_tune_2/T200_beta00.0001_betaT0.02/imputaiton_multiple_165k/'
# dataset_folder = 'datasets/NYISO/dataset_2/mix/'
# dataset_filename = 'mix_test_data_2.npy'

# 2_all
# main_folder = '/home/hchuang/Documents/Project/SSSD_CP/src/'
# result_folder = 'results/NYISO_2_all/T200_beta00.0001_betaT0.02/imputaiton_multiple_70k/'
# dataset_folder = 'datasets/NYISO/dataset_2/all/'
# dataset_filename = 'all_test_data_0.npy'

# 2_all_fine_tune_1
# main_folder = '/home/hchuang/Documents/Project/SSSD_CP/src/'
# result_folder = 'results/NYISO_2_all_fine_tune_1/T200_beta00.0001_betaT0.02/imputaiton_multiple_75k/'
# dataset_folder = 'datasets/NYISO/dataset_2/all/'
# dataset_filename = 'all_test_data_1.npy'

# 2_all_fine_tune_2
# main_folder = '/home/hchuang/Documents/Project/SSSD_CP/src/'
# result_folder = 'results/NYISO_2_all_fine_tune_2/T200_beta00.0001_betaT0.02/imputaiton_multiple_80k/'
# dataset_folder = 'datasets/NYISO/dataset_2/all/'
# dataset_filename = 'all_test_data_2.npy'



# 3_all
# main_folder = '/home/hchuang/Documents/Project/SSSD_CP/src/'
# result_folder = 'results/NYISO_3_all/T200_beta00.0001_betaT0.02/imputaiton_multiple_40k/'
# dataset_folder = 'datasets/NYISO/dataset_3/all/'
# dataset_filename = 'load_array_all_test.npy'

# 3_NYC
# main_folder = '/home/hchuang/Documents/Project/SSSD_CP/src/'
# result_folder = 'results/NYISO_3_NYC/T200_beta00.0001_betaT0.02/imputaiton_multiple/'
# dataset_folder = 'datasets/NYISO/dataset_3/zone/'
# dataset_filename = 'N.Y.C._test.npy'

# 3_MILLWD
main_folder = '/home/hchuang/Documents/Project/SSSD_CP/src/'
result_folder = 'results/NYISO_3_MILLWD/T200_beta00.0001_betaT0.02/imputaiton_multiple/'
dataset_folder = 'datasets/NYISO/dataset_3/zone/'
dataset_filename = 'MILLWD_test.npy'

# 3_CAPITL
# main_folder = '/home/hchuang/Documents/Project/SSSD_CP/src/'
# result_folder = 'results/NYISO_3_CAPITL/T200_beta00.0001_betaT0.02/imputaiton_multiple/'
# dataset_folder = 'datasets/NYISO/dataset_3/zone/'
# dataset_filename = 'CAPITL_test.npy'

In [8]:
pred_data = read_multiple_imputations(main_folder + result_folder, 24)
print(pred_data.shape)
# pred_data_target = pred_data[:,:,:,-24:].reshape(pred_data.shape[1],1,24)
# print(pred_data_target.shape)

file_path = os.path.join(main_folder + result_folder, 'original0.npy')
true_data = np.load(file_path)
print(true_data.shape)
true_data_target = true_data[:,:,-24:]
print(true_data_target.shape)

file_path = os.path.join(main_folder + dataset_folder, dataset_filename)
true_data_o = np.swapaxes(np.load(file_path), axis1=1, axis2=2)
print(true_data_o.shape)

true_data_o_cond = true_data_o[:,:,0:168]
print(true_data_o_cond.shape)
true_data_o_target = true_data_o[:,:,-24:]
print(true_data_o_target.shape)

(10, 73, 1, 24)
(73, 1, 192)
(73, 1, 24)
(73, 1, 192)
(73, 1, 168)
(73, 1, 24)


In [9]:
pred_mean = np.mean(pred_data, axis = 0)
print(pred_mean.shape)

m = np.mean(true_data_o_cond, axis=2).reshape((pred_mean.shape[0], pred_mean.shape[1], 1))
s = np.std(true_data_o_cond, axis=2).reshape((pred_mean.shape[0], pred_mean.shape[1], 1))

pred_mean_o = pred_mean*s + m
print(pred_mean_o.shape)


(73, 1, 24)
(73, 1, 24)


In [10]:
print(f'MAPE: {round(np.mean(abs((true_data_o_target - pred_mean_o))/true_data_o_target)*100,4)}')
print(f'MSE: {round(np.mean(np.square(true_data_target - pred_mean))*1,4)}')

MAPE: 4.2963
MSE: 0.1801


In [12]:
a = np.mean(abs((true_data_o_target - pred_mean_o)/true_data_o_target), axis = 2)*100

In [13]:
a

array([[ 6.96236972],
       [ 4.64923087],
       [ 4.3158702 ],
       [ 3.37546491],
       [ 3.14588432],
       [ 2.06593642],
       [ 1.9258242 ],
       [ 7.5917375 ],
       [ 5.95340866],
       [ 4.62681395],
       [16.0875459 ],
       [ 3.08054554],
       [ 1.77592718],
       [ 3.00267251],
       [ 3.93614476],
       [ 2.31292676],
       [ 1.75877688],
       [ 2.10381575],
       [ 2.16589374],
       [ 4.59400147],
       [ 5.14713543],
       [ 4.47116479],
       [ 4.02793576],
       [ 3.12092458],
       [ 5.61012898],
       [ 4.33934589],
       [ 9.44726431],
       [ 6.24901213],
       [ 2.23708142],
       [ 1.11924964],
       [ 3.10595223],
       [10.21397785],
       [ 5.11704039],
       [ 3.14849355],
       [ 2.14026702],
       [10.65429523],
       [ 2.4693427 ],
       [ 3.86796702],
       [ 2.50636028],
       [ 4.15066096],
       [ 7.34900635],
       [ 4.42164727],
       [ 2.12195315],
       [ 4.43042354],
       [ 3.46263743],
       [ 2

In [19]:
np.where(a > 7)[0]

array([31, 35])

In [67]:
a_1=np.delete(a, np.where(a > 3)[0], None)

In [68]:
np.mean(a_1)

2.3106971467175854

In [8]:
b = np.mean(np.square(true_data_target - pred_mean), axis = 2)

In [9]:
b

array([[0.10712365],
       [0.07521594],
       [0.20686716],
       [0.21641032],
       [0.03020974],
       [0.03626305],
       [0.02188603],
       [0.31249958],
       [0.10018346],
       [0.05234196],
       [0.01995359],
       [0.01967776],
       [0.04203163],
       [0.02984966],
       [0.10260755],
       [0.01055031],
       [0.01927925],
       [0.07953691],
       [0.04061956],
       [0.0146625 ],
       [0.12373526],
       [0.0269018 ],
       [0.0645749 ],
       [0.0482727 ],
       [0.07842322],
       [0.0410951 ],
       [0.17770834],
       [0.01314435],
       [0.01902446],
       [0.01928058],
       [0.03713358],
       [0.6876352 ],
       [0.07915884],
       [0.11015019],
       [0.03124131],
       [0.50380355],
       [0.09481322],
       [0.01125013],
       [0.03982279],
       [0.09678072],
       [0.06815821],
       [0.10238466],
       [0.05614662],
       [0.02234146],
       [0.02878874],
       [0.02322108],
       [0.02539085],
       [0.029

In [7]:
true_data_o_target.shape

(73, 11, 24)

In [8]:
dataset_filename

'load_array_all_test.npy'

In [9]:
if dataset_filename[:3]=='mix':
    print('mix')
    # mix 資料集看單一個 zone
    for i in range(11):
        num = int(true_data_o_target.shape[0]/11)
        zone_obs_num = range(num*(i), num*(i+1))
        print(f'{i} MAPE: {round(np.mean(abs((true_data_o_target[zone_obs_num,:,:] - pred_data_rescale[zone_obs_num,:,:])/true_data_o_target[zone_obs_num,:,:]))*100,4)}')
        # print(f'MSE: {round(np.mean(np.square(true_data_target[:,i,:] - pred_mean[:,i,:]))*1,4)}')
if dataset_filename[:3]=='all' or dataset_filename[11:14]=='all':
    print('all')
    # all 資料集看單一個 zone
    for i in range(11):
        print(f'MAPE: {round(np.mean(abs((true_data_o_target[:,i,:] - pred_data_rescale[:,i,:])/true_data_o_target[:,i,:]))*100,4)}')
        print(f'MSE: {round(np.mean(np.square(true_data_target[:,i,:] - pred_mean[:,i,:]))*1,4)}')

all
MAPE: 2.5456
MSE: 0.0909
MAPE: 2.6123
MSE: 0.0976
MAPE: 2.6834
MSE: 0.0624
MAPE: 2.674
MSE: 0.0803
MAPE: 2.7752
MSE: 0.0916
MAPE: 2.8252
MSE: 0.0676
MAPE: 3.5359
MSE: 0.1479
MAPE: 4.2743
MSE: 0.1815
MAPE: 2.0187
MSE: 0.0385
MAPE: 2.0706
MSE: 0.2648
MAPE: 2.2605
MSE: 0.0919


# Select which pkl

compare each imputation generated from different pkl

In [25]:
main_folder = '/home/hchuang/Documents/Project/SSSD_CP/src/'
result_folder = 'results/NYISO_mix/T200_beta00.0001_betaT0.02/compare_pkl_test_data_1/'
dataset_folder = 'datasets/NYISO/dataset_2/mix/'
dataset_filename = 'mix_test_data_1.npy'

pred_data = read_multiple_imputations(main_folder + result_folder, 24)

file_path = os.path.join(main_folder + result_folder, 'original0.npy')
true_data = np.load(file_path)
true_data_target = true_data[:,:,-24:]

file_path = os.path.join(main_folder + dataset_folder, dataset_filename)
true_data_o = np.swapaxes(np.load(file_path), axis1=1, axis2=2)

true_data_o_cond = true_data_o[:,:,0:168]
true_data_o_target = true_data_o[:,:,-24:]

m = np.mean(true_data_o_cond, axis=2).reshape((true_data_o_cond.shape[0], 1, 1))
s = np.std(true_data_o_cond, axis=2).reshape((true_data_o_cond.shape[0], 1, 1))

# for i in range(17):
for i in [0,1,2,3,9,13,14,15,16]:    
    num = i+1
    file_name = str(num) + '0_imputation0.npy'
    file_path = os.path.join(main_folder + result_folder, file_name)
    pred_each = np.load(file_path)
    pred_each = pred_each[:,:,-24:]
    # pred_each = pred_data[i,:,:,:].reshape(pred_data.shape[1],pred_data.shape[2],pred_data.shape[3])

    pred_data_rescale = pred_each*s + m
    print(f'MAPE: {np.mean(abs((true_data_o_target - pred_data_rescale)/true_data_o_target))*100}')


MAPE: 3.5904037804097957
MAPE: 2.898665800129504
MAPE: 2.8213715364070935
MAPE: 2.744114768133877
MAPE: 2.634120765526516
MAPE: 2.580837563737816
MAPE: 2.5700620774130165
MAPE: 2.5964068438556995
MAPE: 2.5493624462224145


# mix data 對應到 zone


In [27]:
dataset_folder = 'datasets/NYISO/dataset_2/mix/'
dataset_filename = 'mix_test_data_0_stdd.npy'
file_path = os.path.join(main_folder + dataset_folder, dataset_filename)
true_data = np.swapaxes(np.load(file_path), axis1=1, axis2=2)
print(true_data.shape)

(22, 1, 192)


In [38]:
dataset_folder = 'datasets/NYISO/dataset_2/zone/'
dataset_filename = 'WEST_test_data_0_stdd.npy'
file_path = os.path.join(main_folder + dataset_folder, dataset_filename)
zone_true_data = np.swapaxes(np.load(file_path), axis1=1, axis2=2)
print(zone_true_data.shape)

(2, 1, 192)


In [39]:
print(true_data[20:22,0,0])
print(zone_true_data[0:2,0,0])

[-0.84438167 -0.98628458]
[-0.84438167 -0.98628458]


In [12]:
2%1

0

In [24]:
n_iter = 0
while n_iter < 3:
    # print(n_iter)
    print(n_iter)
    for i in range(2):
        i+i
        # print(n_iter)
    n_iter += 1
    

0
1
2
